<div class="text_center", style="font-family: 'Times New Roman', Times, serif;
                                 text-align: center;
                                 font-weight: normal;
                                 font-size: 18px;
                                 padding: 0px 100px 0px 0px;">
    <span style="font-size: 15px; font-style: italic;">
        Partial Differential Equations with Applications in Physics and Industry
    </span>
    <hr>
    <br>
    <br> 
    <span style="font-size: 40px;"> Wave equation for vibrating membrane </span>
    <br>
    <br>
    <br>
    Marcin Kostrzewa, &nbsp;Marcin Miśkiewicz
</div>

<div class="text", style="font-family: 'Times New Roman', Times, serif;
                          font-weight: normal;
                          text-align: justify;
                          font-size: 18px;
                          padding: 0px 100px 0px 0px;">
    <h3>Abstract</h3>
    <hr>
    Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.
</div>

<div class="text", style="font-family: 'Times New Roman', Times, serif;
                          font-weight: normal;
                          text-align: justify;
                          font-size: 18px;
                          padding: 0px 100px 0px 0px;">
    <h3>Results</h3>
    <hr>
    Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.
</div>

In [27]:
using Plots, Measures, LinearAlgebra, Logging
Logging.disable_logging(Logging.Info); # disable ugly messages after generating animations

In [28]:

function grid(; Δx::Float64, Δy::Float64, Domain::NamedTuple)
    (A, B, C, D) = Domain
    xs = A:Δx:B
    ys = C:Δy:D
    nx, ny = length(xs), length(ys)
    xx = xs' .* ones(ny)
    yy = ys  .* ones(nx)'
    return (nx, ny, xx, yy)
end


function explicit_finite_difference(; Δx::Float64, 
                                      Δy::Float64, 
                                      Δt::Float64, 
                                      c::Float64, 
                                      N::Int64,
                                      IC::Function, 
                                      BCs::NamedTuple, 
                                      grid_params::Tuple)

    (nx, ny, xx, yy) = grid_params
    (u_Ay, u_By, u_xC, u_xD) = BCs

    rx = c * Δt / Δx
    ry = c * Δt / Δy
    r = rx + ry

    if r < 1/√2
        (@warn "CFL stability condition is not met! (r = $r < 1/√2)")
    end

    u = zeros(ny, nx, N)
    u[:,  :, 1] = IC.(xx, yy)
    u[:,  :, 2] = IC.(xx, yy)
    u[1,  :, 1] .= u_Ay
    u[:,  1, 1] .= u_xC
    u[ny, :, 1] .= u_By
    u[:, nx, 1] .= u_xD
    i = collect(2:nx-1)
    j = collect(2:ny-1)

    for t in 3:N 
        u[i, j, t] = (u[i, j, t.-1] * (2 - 2*(rx^2 + ry^2)) 
                    .+ rx^2 * (u[i.+1, j, t.-1] .+ u[i.-1, j, t.-1]) 
                    .+ ry^2 * (u[i, j.+1, t.-1] .+ u[i, j.-1, t.-1]) 
                    .- u[i, j, t.-2])
                        
    end
    return u
end

explicit_finite_difference (generic function with 1 method)

In [29]:
IC = (x, y) -> exp(-x^2 - y^2)

BCs = (  # Dirichlet Boundary Conditions
    u_Ay = 0,
    u_By = 0,
    u_xC = 0,
    u_xD = 0,
)

Domain = (   # Rectangle
    A = -5,  # x min
    B =  5,  # x max
    C = -5,  # y min
    D =  5,  # y max
)

Δx = 0.1 
Δy = 0.1
Δt = 0.1
c  = 0.7
N  = 700

grid_params = grid(Δx=Δx, Δy=Δy, Domain=Domain)
(nx, ny, xx, yy) = grid_params
u = explicit_finite_difference(Δx=Δx, Δy=Δy, Δt=Δt, c=c, N=N, IC=IC, BCs=BCs, grid_params=grid_params);

In [30]:
colormap = cgrad([:midnightblue, :purple, :purple, :skyblue, :cyan, :cyan, :white])
frames = size(u)[3] * 0.5 |> round |> Int
camera_angles = LinRange(0, 360, frames)
l = @layout [a{0.6w} b]
xs, ys = xx[1, :], yy[:, 1]
SIZE = (1000, 500)


an = @animate for t in 1:frames

    p1 = surface(xx, yy, u[:, :, t],
                 zlims=(extrema(u)),
                 colormap=colormap,
                 colorbar=false,
                 zlabel="u")

    p2 = heatmap(xs, ys, u[:, :, t],
                  colormap=colormap)

    plot(p1, p2, 
        layout=l, 
        xlabel="x",
        ylabel="y",
        clims=extrema(u),
        xlims=extrema(xs),
        ylims=extrema(ys), 
        camera=(camera_angles[t], 30), 
        plot_title="t = $(round(t*Δt, digits=2))",
        aspect_ratio=:equal, 
        size=SIZE,
        link=:all)
end

gif(an, "./media/test.gif", fps = 25);

<img src="./media/test.gif">